In [ ]:
import collections
import time

class CSPSolver:
    """
    Constraint Satisfaction Problem (CSP) for map coloring.

    Attributes:
        variables (list): A list of regions (variables).
        domains (dict): A dictionary mapping each variable to its domain of possible colors.
        constraints (dict): An adjacency list representing the graph of regions.
        stats (dict): A dictionary to keep track of performance metrics.
    """

    def __init__(self, variables, domains, constraints):
        """Initializes the CSP Solver."""
        self.variables = variables
        self.domains = domains
        self.constraints = constraints
        self.stats = {'backtracks': 0, 'assignments': 0}

    def solve(self, use_mrv=False, use_lcv=False, use_forward_checking=False, use_ac3=False):
        """
        Public method to solve the CSP.

        Args:
            use_mrv (bool): Whether to use the Minimum Remaining Values heuristic.
            use_lcv (bool): Whether to use the Least Constraining Value heuristic.
            use_forward_checking (bool): Whether to use forward checking.
            use_ac3 (bool): Whether to use the AC-3 algorithm for constraint propagation.

        Returns:
            A dictionary representing the solution, or None if no solution is found.
        """
        self.stats = {'backtracks': 0, 'assignments': 0}
        
        # AC-3 can be run once at the start to pre-process the domains
        if use_ac3:
            if not self.ac3():
                print("AC-3 determined no solution is possible from the start.")
                return None

        solution = self.backtrack({}, use_mrv, use_lcv, use_forward_checking, use_ac3)
        return solution

    def backtrack(self, assignment, use_mrv, use_lcv, use_forward_checking, use_ac3):
        """
        The core recursive backtracking algorithm.

        Args:
            assignment (dict): The current assignment of colors to variables.
            (See solve method for other args)

        Returns:
            A complete assignment (solution) or None (failure).
        """
        # If assignment is complete, we have found a solution
        if len(assignment) == len(self.variables):
            return assignment

        # Select the next variable to assign
        var = self.select_unassigned_variable(assignment, use_mrv)
        
        # Order the values for the selected variable
        ordered_values = self.order_domain_values(var, assignment, use_lcv)

        for value in ordered_values:
            self.stats['assignments'] += 1
            if self.is_consistent(var, value, assignment):
                # Add {var: value} to assignment
                assignment[var] = value
                
                # --- Inference (Filtering) ---
                # Keep a copy of original domains to restore on backtrack
                original_domains = {v: list(d) for v, d in self.domains.items()}
                
                inferences_ok = True
                if use_forward_checking:
                    inferences_ok = self.forward_checking(var, value, assignment)
                elif use_ac3:
                    # For AC-3 during search, we only need to check arcs from neighbors of var
                    inferences_ok = self.ac3(arcs_to_check=[(neighbor, var) for neighbor in self.constraints[var]])

                if inferences_ok:
                    result = self.backtrack(assignment, use_mrv, use_lcv, use_forward_checking, use_ac3)
                    if result is not None:
                        return result

                # Restore domains if inference or recursive call failed
                self.domains = original_domains

            # If assignment is inconsistent or leads to failure, remove it
            if var in assignment:
                del assignment[var]

        self.stats['backtracks'] += 1
        return None

    def is_consistent(self, var, value, assignment):
        """
        Check if a value for a variable is consistent with the current assignment.
        A value is consistent if it does not conflict with any assigned neighbors.
        """
        for neighbor in self.constraints[var]:
            if neighbor in assignment and assignment[neighbor] == value:
                return False
        return True

    # --- Heuristics ---

    def select_unassigned_variable(self, assignment, use_mrv):
        """
        Selects the next variable to assign.
        If MRV is enabled, it uses the Minimum Remaining Values heuristic.
        Otherwise, it just picks the next unassigned variable in order.
        """
        unassigned = [v for v in self.variables if v not in assignment]
        
        if use_mrv:
            # MRV: Choose the variable with the fewest legal values in its domain
            # As a tie-breaker, use the degree heuristic (most constrained variable)
            return min(unassigned, key=lambda var: (len(self.domains[var]), -len(self.constraints[var])))
        else:
            return unassigned[0]

    def order_domain_values(self, var, assignment, use_lcv):
        """
        Orders the values in the domain of a variable.
        If LCV is enabled, it uses the Least Constraining Value heuristic.
        Otherwise, it returns the domain values in their default order.
        """
        if use_lcv:
            # LCV: Prefer the value that rules out the fewest choices for neighboring variables
            def count_conflicts(value):
                conflicts = 0
                for neighbor in self.constraints[var]:
                    if neighbor not in assignment and value in self.domains[neighbor]:
                        conflicts += 1
                return conflicts
            return sorted(self.domains[var], key=count_conflicts)
        else:
            return self.domains[var]

    # --- Filtering / Inference Techniques ---

    def forward_checking(self, var, value, assignment):
        """
        Implements forward checking.
        When a variable `var` is assigned `value`, this function removes `value`
        from the domain of all unassigned neighbors of `var`.
        """
        for neighbor in self.constraints[var]:
            if neighbor not in assignment:
                if value in self.domains[neighbor]:
                    self.domains[neighbor].remove(value)
                if not self.domains[neighbor]: # Domain wipeout
                    return False
        return True

    def ac3(self, arcs_to_check=None):
        """
        Implements the AC-3 algorithm for arc consistency.
        If `arcs_to_check` is None, it initializes the queue with all arcs.
        Otherwise, it uses the provided list of arcs (useful during search).
        """
        queue = collections.deque(arcs_to_check or self.get_all_arcs())
        
        while queue:
            xi, xj = queue.popleft()
            if self.revise(xi, xj):
                if not self.domains[xi]: # Domain wipeout
                    return False
                # If a value was removed from xi's domain, we need to re-check
                # all arcs pointing to xi.
                for xk in self.constraints[xi]:
                    if xk != xj:
                        queue.append((xk, xi))
        return True

    def get_all_arcs(self):
        """Helper function to get all arcs for the initial AC-3 run."""
        arcs = []
        for var in self.variables:
            for neighbor in self.constraints[var]:
                arcs.append((var, neighbor))
        return arcs

    def revise(self, xi, xj):
        """
        Helper function for AC-3. Returns true if we remove a value from domain of xi.
        An arc (xi, xj) is consistent if for every value x in domain(xi),
        there is some allowed value y in domain(xj).
        """
        revised = False
        for x in self.domains[xi][:]:
            # Check if there is any value y in domain of xj that satisfies the constraint
            if not any(x != y for y in self.domains[xj]):
                self.domains[xi].remove(x)
                revised = True
        return revised



In [2]:

def define_australia_map():
    """Defines the map of Australia as a CSP."""
    variables = ['WA', 'NT', 'SA', 'Q', 'NSW', 'V', 'T']
    colors = ['Red', 'Green', 'Blue']
    domains = {var: list(colors) for var in variables}
    
    # Constraints are defined by shared borders
    constraints = {
        'WA': ['NT', 'SA'],
        'NT': ['WA', 'SA', 'Q'],
        'SA': ['WA', 'NT', 'Q', 'NSW', 'V'],
        'Q': ['NT', 'SA', 'NSW'],
        'NSW': ['Q', 'SA', 'V'],
        'V': ['SA', 'NSW'],
        'T': []
    }
    return variables, domains, constraints

def print_results(name, solution, stats, duration):
    """Helper function to print the results of a solver run."""
    print(f"--- {name} ---")
    if solution:
        print(f"Solution found in {duration:.5f} seconds:")
        # Sort for consistent output
        sorted_solution = dict(sorted(solution.items()))
        print(sorted_solution)
    else:
        print(f"No solution found. Searched for {duration:.5f} seconds.")
    print(f"Assignments: {stats['assignments']:,}")
    print(f"Backtracks: {stats['backtracks']:,}")
    print("-" * (len(name) + 8) + "\n")


In [3]:


if __name__ == '__main__':
    # --- Run and Compare Different Strategies ---

    # 1. Simple Backtracking
    variables, domains, constraints = define_australia_map()
    solver = CSPSolver(variables, domains, constraints)
    start_time = time.time()
    solution = solver.solve()
    duration = time.time() - start_time
    print_results("1. Simple Backtracking", solution, solver.stats, duration)

    # 2. Backtracking with MRV Heuristic
    variables, domains, constraints = define_australia_map()
    solver = CSPSolver(variables, domains, constraints)
    start_time = time.time()
    solution = solver.solve(use_mrv=True)
    duration = time.time() - start_time
    print_results("2. Backtracking + MRV", solution, solver.stats, duration)

    # 3. Backtracking with Forward Checking and MRV
    variables, domains, constraints = define_australia_map()
    solver = CSPSolver(variables, domains, constraints)
    start_time = time.time()
    solution = solver.solve(use_mrv=True, use_forward_checking=True)
    duration = time.time() - start_time
    print_results("3. Backtracking + Forward Checking + MRV", solution, solver.stats, duration)

    # 4. Backtracking with AC-3, MRV, and LCV (Full Power)
    variables, domains, constraints = define_australia_map()
    solver = CSPSolver(variables, domains, constraints)
    start_time = time.time()
    solution = solver.solve(use_mrv=True, use_lcv=True, use_ac3=True)
    duration = time.time() - start_time
    print_results("4. Backtracking + AC-3 + MRV + LCV", solution, solver.stats, duration)



--- 1. Simple Backtracking ---
Solution found in 0.00000 seconds:
{'NSW': 'Green', 'NT': 'Green', 'Q': 'Red', 'SA': 'Blue', 'T': 'Red', 'V': 'Red', 'WA': 'Red'}
Assignments: 11
Backtracks: 0
------------------------------

--- 2. Backtracking + MRV ---
Solution found in 0.00000 seconds:
{'NSW': 'Green', 'NT': 'Green', 'Q': 'Blue', 'SA': 'Red', 'T': 'Red', 'V': 'Blue', 'WA': 'Blue'}
Assignments: 15
Backtracks: 0
-----------------------------

--- 3. Backtracking + Forward Checking + MRV ---
Solution found in 0.00000 seconds:
{'NSW': 'Green', 'NT': 'Green', 'Q': 'Blue', 'SA': 'Red', 'T': 'Red', 'V': 'Blue', 'WA': 'Blue'}
Assignments: 7
Backtracks: 0
------------------------------------------------

--- 4. Backtracking + AC-3 + MRV + LCV ---
Solution found in 0.00000 seconds:
{'NSW': 'Green', 'NT': 'Green', 'Q': 'Blue', 'SA': 'Red', 'T': 'Red', 'V': 'Blue', 'WA': 'Blue'}
Assignments: 15
Backtracks: 0
------------------------------------------

